# PREDICCION DIXON COLES

In [1]:
import pandas as pd
from scipy.stats import poisson
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')
pd.options.display.max_columns = None #quitar limite de visualizacion de columnas
import matplotlib.pyplot as plt
from scipy.stats import poisson,skellam
from scipy.optimize import minimize
import math

# CREAMOS UN MODELO PARA PREDICIR UNA TEMPORADA EN FUNCION DE LAS CUATRO ANTERIORES
OBJETIVO: predecir resultados temporada 2021/2022

# DIXON-COLES PARA LA CORRECCION DE LA SUBESTIMACION EN LOS EMPATES


Los resultados bajos (0.0, 1.0, 0.1, 1.1) estan subestimados con el modelo de Poisson Bivariante

In [2]:
resultados = pd.read_csv('resultados_93_23_la_liga.csv')
resultados_previos = resultados[((resultados.Temporada == '20/21') | (resultados.Temporada == '19/20') | (resultados.Temporada == '18/19') | (resultados.Temporada == '17/18'))]
resultados_a_predecir = pd.read_csv('resultados_a_predecir_21_22_i_BP.csv')

In [3]:
resultados_previos.head()

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
450,20/21,La Liga,2021-05-23,Granada,Getafe,0,0,D
451,20/21,La Liga,2021-05-23,Sevilla,Alaves,1,0,H
452,20/21,La Liga,2021-05-22,Real Madrid,Villarreal,2,1,H
453,20/21,La Liga,2021-05-22,Osasuna,Sociedad,0,1,A
454,20/21,La Liga,2021-05-22,Huesca,Valencia,0,0,D


In [4]:
resultados_a_predecir.head()

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,FTR_predict_i,FTR_predict_BP
0,21/22,La Liga,2022-05-22,Sevilla,Ath Bilbao,1,0,H,H,H
1,21/22,La Liga,2022-05-22,Barcelona,Villarreal,0,2,A,H,H
2,21/22,La Liga,2022-05-22,Osasuna,Mallorca,0,2,A,H,H
3,21/22,La Liga,2022-05-22,Granada,Espanol,0,0,D,H,H
4,21/22,La Liga,2022-05-22,Alaves,Cadiz,0,1,A,A,H


In [5]:
print(len(resultados_previos),'partidos disputados')

1520 partidos disputados


In [6]:
stats = pd.read_csv('stats_4temporadas_17_21_i_BP.csv')
stats

,Equipo,VL,DL,EL,VV,DV,EV,FAL_i,FAL_BP,FDL_i,FDL_BP,FAV_i,FAV_BP,FDV_i,FDV_BP
0,La Coruna,4,9,6,2,12,5,-0.202176,0.582959,0.556604,-0.059772,-0.245283,0.319366,0.559383,0.105871
1,Malaga,4,12,3,1,16,2,-0.492294,0.111159,0.273585,-0.275474,-0.528302,-0.166264,0.233001,-0.146253
2,Las Palmas,4,13,2,1,13,5,-0.456029,0.184323,0.792453,0.064467,-0.575472,-0.245288,0.305530,-0.087078
3,Rayo Vallecano,5,8,6,3,14,2,-0.093382,0.748688,0.509434,-0.126989,-0.245283,0.359977,0.378060,-0.034611
4,Girona,11,18,9,12,16,10,-0.220308,0.545660,0.179245,-0.333111,0.037736,0.532396,0.124207,-0.484667
5,Mallorca,8,8,3,1,15,3,-0.093382,0.729155,0.037736,-0.520686,-0.292453,0.300086,0.559383,0.070508
6,Leganes,21,19,17,10,33,14,-0.335147,0.167875,-0.040881,-0.467636,-0.276730,0.220313,0.015413,-0.549402
7,Espanol,21,21,15,10,28,19,-0.238441,0.459169,0.084906,-0.134450,-0.245283,0.284707,-0.020852,-0.442848
8,Villarreal,33,21,22,28,30,18,0.169538,0.778934,0.132075,-0.439703,0.179245,0.947849,-0.102448,-0.585529
9,Valencia,39,10,27,22,34,20,0.115141,0.948506,-0.209906,-0.604917,0.049528,0.431422,0.015413,-0.160728


In [7]:
import numpy as np
    
def likelihood(FAL, FDL, FAV, FDV, GL, GV, rho):
    a = np.exp(FAL + FDV)
    b = np.exp(FAV + FDL)
    return np.log(rho_correccion(GL, GV, a, b, rho))-a+GL*np.log(a)-b+GV*np.log(b)

def rho_correccion(x, y, a, b, rho):
    if x==0 and y==0: return 1 - (a * b * rho)
    elif x==0 and y==1: return 1 + (a * rho)
    elif x==1 and y==0: return 1 + (b * rho)
    elif x==1 and y==1: return 1 - rho
    else: return 1.0

def sumatorio_likelihoods(params):
    temp = []
    for i, teamA in enumerate(stats.Equipo):
        for j, teamB in enumerate(stats.Equipo):
            if teamA != teamB and len(resultados_previos[((resultados_previos.HomeTeam == teamA) & (resultados_previos.AwayTeam == teamB))]) != 0:
                FAL = params[i]
                FDL = params[len(stats)+i]
                FAV = params[2*len(stats)+j]
                FDV = params[3*len(stats)+j]
                rho = params[-1]
                GL = resultados_previos[((resultados_previos.HomeTeam == teamA) & (resultados_previos.AwayTeam == teamB))].FTHG.iloc[0]
                GV = resultados_previos[((resultados_previos.HomeTeam == teamA) & (resultados_previos.AwayTeam == teamB))].FTAG.iloc[0]
                #print('GL:', GL, teamA, 'FAL:', FAL, 'FDL:', FDL, '\nGV:', GV, teamB, 'FAV:',  FAV, 'FDV:', FDV,'\n\n')
                temp.append(likelihood(FAL, FDL, FAV, FDV, GL, GV, rho))
    return -sum(temp)

Fijamos Rho inicial = 0

In [8]:
params = list(stats.FAL_BP) + list(stats.FDL_BP) + list(stats.FAV_BP) + list(stats.FDV_BP) + [0]
len(params)

113

In [9]:
from scipy.optimize import minimize 
from datetime import datetime

start = datetime.now()
print(start)
resultados_minimize = minimize(sumatorio_likelihoods, params, method='SLSQP')

stop = datetime.now()
print(stop)
print(f'Time taken to run: {stop - start}')

2022-10-06 20:17:45.453166
2022-10-06 20:39:40.425602
Time taken to run: 0:21:54.972436


In [10]:
resultados_minimize.x[-1]

-0.03182734982038434

In [11]:
stats.insert(9, 'FAL_DC', resultados_minimize.x[:len(stats)])
stats.insert(12, 'FDL_DC', resultados_minimize.x[len(stats):len(stats)*2])
stats.insert(15, 'FAV_DC', resultados_minimize.x[len(stats)*2:len(stats)*3])
stats.insert(18, 'FDV_DC', resultados_minimize.x[len(stats)*3:-1])

In [12]:
stats

,Equipo,VL,DL,EL,VV,DV,EV,FAL_i,FAL_BP,FAL_DC,FDL_i,FDL_BP,FDL_DC,FAV_i,FAV_BP,FAV_DC,FDV_i,FDV_BP,FDV_DC
0,La Coruna,4,9,6,2,12,5,-0.202176,0.582959,0.584544,0.556604,-0.059772,-0.059537,-0.245283,0.319366,0.321468,0.559383,0.105871,0.107769
1,Malaga,4,12,3,1,16,2,-0.492294,0.111159,0.107530,0.273585,-0.275474,-0.275922,-0.528302,-0.166264,-0.172214,0.233001,-0.146253,-0.144866
2,Las Palmas,4,13,2,1,13,5,-0.456029,0.184323,0.180841,0.792453,0.064467,0.060433,-0.575472,-0.245288,-0.245714,0.305530,-0.087078,-0.085434
3,Rayo Vallecano,5,8,6,3,14,2,-0.093382,0.748688,0.749722,0.509434,-0.126989,-0.129432,-0.245283,0.359977,0.356496,0.378060,-0.034611,-0.038065
4,Girona,11,18,9,12,16,10,-0.220308,0.545660,0.546492,0.179245,-0.333111,-0.331966,0.037736,0.532396,0.532981,0.124207,-0.484667,-0.484966
5,Mallorca,8,8,3,1,15,3,-0.093382,0.729155,0.725719,0.037736,-0.520686,-0.523266,-0.292453,0.300086,0.292034,0.559383,0.070508,0.070845
6,Leganes,21,19,17,10,33,14,-0.335147,0.167875,0.171665,-0.040881,-0.467636,-0.467260,-0.276730,0.220313,0.218467,0.015413,-0.549402,-0.547976
7,Espanol,21,21,15,10,28,19,-0.238441,0.459169,0.459244,0.084906,-0.134450,-0.132937,-0.245283,0.284707,0.284555,-0.020852,-0.442848,-0.442191
8,Villarreal,33,21,22,28,30,18,0.169538,0.778934,0.779084,0.132075,-0.439703,-0.439606,0.179245,0.947849,0.949391,-0.102448,-0.585529,-0.583327
9,Valencia,39,10,27,22,34,20,0.115141,0.948506,0.945899,-0.209906,-0.604917,-0.607108,0.049528,0.431422,0.430720,0.015413,-0.160728,-0.162133


In [13]:
stats.to_csv('stats_4temporadas_17_21_i_BP_DC.csv', index = False)

In [14]:
def simular_partido(stats, local, visitante, modelo='i'):
    
    FAL, FDL, FAV, FDV = 'FAL_'+ modelo, 'FDL_'+ modelo, 'FAV_'+ modelo, 'FDV_'+ modelo
    
    FAL = stats[stats.Equipo == local][FAL].iloc[0]
    FDL = stats[stats.Equipo == local][FDL].iloc[0]
    FAV = stats[stats.Equipo == visitante][FAV].iloc[0]
    FDV = stats[stats.Equipo == visitante][FDV].iloc[0]
        
    objetivo_esperado_local = math.exp(FAL + FDV)
    objetivo_esperado_visitante = math.exp(FAV + FDL)
    #print('EN EL MODELO', modelo, ':\n--------------------------')
    #print('Se espera que el', local, 'marque', round(objetivo_esperado_local,3), 'goles contra el', visitante)
    #print('Se espera que el', visitante, 'marque', round(objetivo_esperado_visitante,3), 'goles contra el', local,'\n')
    team_local_poisson = [poisson.pmf(i, objetivo_esperado_local) for i in range(10)]
    team_visitante_poisson = [poisson.pmf(i, objetivo_esperado_visitante) for i in range(10)]
    partido = np.outer(team_local_poisson, team_visitante_poisson)
    tabla = pd.DataFrame(partido)
    victoria_local = np.sum(np.tril(partido, -1))
    empate = np.sum(np.diag(partido))
    victoria_visit = np.sum(np.triu(partido, 1))

    #print('Prob implicita victoria local:', round(victoria_local*100,2), '%')
    #print('Prob implicita Empate:', round(empate*100, 2), '%')
    #print('Prob implicita victoria visitante:', round(victoria_visit*100,2), '%\n')
    
    res = max(victoria_local, empate, victoria_visit)
    if res ==  victoria_local: FTR = 'H'
    if res == empate: FTR = 'D'
    if res == victoria_visit: FTR = 'A'
    
    return FTR, tabla

local = 'Sevilla'
visitante = 'Ath Bilbao'
FTR, simulacion_resultados_final = simular_partido(stats, local, visitante, modelo='DC')

In [15]:
#Sevilla vs Ath Bilbao
simulacion_resultados_final

,0,1,2,3,4,5,6,7,8,9
0,0.154824,0.065578,0.013888,1.960902e-03,2.076438e-04,1.759025e-05,1.241778e-06,7.513961e-08,3.978341e-09,1.872331e-10
1,0.223240,0.094557,0.020026,2.827423e-03,2.994014e-04,2.536337e-05,1.790518e-06,1.083437e-07,5.736366e-09,2.699712e-10
2,0.160945,0.068171,0.014438,2.038429e-03,2.158533e-04,1.828571e-05,1.290873e-06,7.811036e-08,4.135631e-09,1.946356e-10
3,0.077356,0.032765,0.006939,9.797364e-04,1.037462e-04,8.788715e-06,6.204363e-07,3.754242e-08,1.987721e-09,9.354830e-11
4,0.027885,0.011811,0.002501,3.531703e-04,3.739790e-05,3.168110e-06,2.236516e-07,1.353309e-08,7.165231e-10,3.372180e-11
5,0.008041,0.003406,0.000721,1.018472e-04,1.078480e-05,9.136190e-07,6.449662e-08,3.902671e-09,2.066308e-10,9.724686e-12
6,0.001932,0.000819,0.000173,2.447556e-05,2.591765e-06,2.195577e-07,1.549960e-08,9.378763e-10,4.965679e-11,2.337003e-12
7,0.000398,0.000169,0.000036,5.041611e-06,5.338662e-07,4.522572e-08,3.192694e-09,1.931890e-10,1.022858e-11,4.813888e-13
8,0.000072,0.000030,0.000006,9.086868e-07,9.622265e-08,8.151366e-09,5.754429e-10,3.481988e-11,1.843573e-12,8.676426e-14
9,0.000011,0.000005,0.000001,1.455816e-07,1.541593e-08,1.305939e-09,9.219228e-11,5.578527e-12,2.953607e-13,1.390059e-14


In [16]:
def resultados_mas_probables(tabla):
    aux = tabla.copy()
    tops =[]
    df = pd.DataFrame(columns=['local', 'visitante', 'prob (%)'])
    prob = 0
    for _ in range(10):
        top=[0,0,prob]
        for col in range(aux.shape[1]):
            for fil in range(aux.shape[0]):
                prob = aux.iloc[fil,col]
                if prob > top[-1]: top = [fil,col,prob]
        aux.iloc[top[0], top[1]]=0
        tops.append(top)
        df = df.append({'local': top[0], 'visitante':top[1], 'prob (%)':round(top[-1]*100,2)}, ignore_index=True)
        df.local = df.local.astype('int64')
        df.visitante = df.visitante.astype('int64')
        
    return df, (df.local.iloc[0], df.visitante.iloc[0])

top10_predict, resultado_predict = resultados_mas_probables(simulacion_resultados_final)

In [17]:
top10_predict

,local,visitante,prob (%)
0,1,0,22.32
1,2,0,16.09
2,0,0,15.48
3,1,1,9.46
4,3,0,7.74
5,2,1,6.82
6,0,1,6.56
7,3,1,3.28
8,4,0,2.79
9,1,2,2.0


In [18]:
def predict(resultados, stats, modelo):
    resultados_predict = []
    for i in range(len(resultados)):
        local = resultados.HomeTeam.iloc[i]
        visitante = resultados.AwayTeam.iloc[i]
        FTR = simular_partido(stats, local, visitante, modelo)[0]
        #print('\n')
        resultados_predict.append(FTR)
    resultados_a_predecir['FTR_predict_'+modelo] = resultados_predict
    return resultados_a_predecir
predict(resultados_a_predecir, stats, 'DC')

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,FTR_predict_i,FTR_predict_BP,FTR_predict_DC
0,21/22,La Liga,2022-05-22,Sevilla,Ath Bilbao,1,0,H,H,H,H
1,21/22,La Liga,2022-05-22,Barcelona,Villarreal,0,2,A,H,H,H
2,21/22,La Liga,2022-05-22,Osasuna,Mallorca,0,2,A,H,H,H
3,21/22,La Liga,2022-05-22,Granada,Espanol,0,0,D,H,H,H
4,21/22,La Liga,2022-05-22,Alaves,Cadiz,0,1,A,A,H,H
...,...,...,...,...,...,...,...,...,...,...,...
375,21/22,La Liga,2021-08-14,Osasuna,Espanol,0,0,D,A,H,H
376,21/22,La Liga,2021-08-14,Alaves,Real Madrid,1,4,A,A,A,A
377,21/22,La Liga,2021-08-14,Mallorca,Betis,1,1,D,A,H,H
378,21/22,La Liga,2021-08-14,Cadiz,Levante,1,1,D,A,A,A


In [20]:
predict(resultados_a_predecir, stats, 'DC').to_csv('resultados_a_predecir_21_22_i_BP_DC.csv', index=False)

In [19]:
def fiabilidad(resultados, modelo):
    aciertos = 0
    home = 0
    draw = 0
    away = 0
    for i in range(len(resultados)):
        if resultados.FTR.iloc[i] == resultados['FTR_predict_'+modelo].iloc[i]:
            if resultados.FTR.iloc[i] == 'H': home+=1 
            if resultados.FTR.iloc[i] == 'D': draw+=1
            if resultados.FTR.iloc[i] == 'A': away+=1
            aciertos += 1

    print('----------------------------------------------------')
    print('FIABILIDAD MODELO', modelo,':')
    print('----------------------------------------------------')
    print(aciertos, '/', len(resultados), 'resultados acertados \t\t--->',round(aciertos/len(resultados)*100,2), '%')
    print('----------------------------------------------------')
    print(home + away, '/', len(resultados[resultados.FTR != 'D']), 'victorias/derrotas acertadas \t--->',round((home+away)/len(resultados[resultados.FTR != 'D'])*100,2), '%')
    print(draw, '/', len(resultados[resultados.FTR == 'D']), 'empates acertados \t\t--->',round(draw/len(resultados[resultados.FTR == 'D'])*100,2), '%')
fiabilidad(predict(resultados_a_predecir, stats, 'DC'), 'DC')

----------------------------------------------------
FIABILIDAD MODELO DC :
----------------------------------------------------
192 / 380 resultados acertados 		---> 50.53 %
----------------------------------------------------
191 / 269 victorias/derrotas acertadas 	---> 71.0 %
1 / 111 empates acertados 		---> 0.9 %
